#Data/Object types
<ul>

  <li>
    <a>MirroredStrategy</a>: This object is used to handle distribution when using distributed training with keras
  </li>
  <li>
    <a>OptionsDataset</a>: A dataset obtained from tensorflow mnist:

    datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
    mnist_train, mnist_test = datasets['train'], datasets['test']

  </li>
  <li>
    <a>BatchDataset</a>: When you shuffle and batch an optionsDataset:

    train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  </li>
  <li>
    <a>CachedDataset</a>: When you shuffle, batch and cache an optionsDataset:
    
    train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache()
  </li>

</ul>


#Some useful functions/code snippets:

<ul> 
  <li>
    <a>strategy.num_replicas_in_sync</a>: returns the ammount of devices connected for distributed training

    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

  </li>

  <li>
    <a>model.save </a>, saves the model (including training data) to a path

    path = 'saved_model/'
    model.save(path, save_format='tf')

  Note: To load the model use: <a>tf.keras.models.load_model(path)</a>

    #Load the model without strategy.scope.
    unreplicated_model = tf.keras.models.load_model(path)

    unreplicated_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy'])

    eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)
    print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))
  </li>
  <li>
  <a>tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE)</a> 

      Basically transforms the data into a proper tensor to be handled.
      It returns a BatchDataset dataType, it also returns shape (None, singe_instance_train_data_shape)
  </li>
  <li>
  <a>model.predict(data_to_predict_from)</a> 

      returns an array of length (data_to_predict_from) and of the shape you indicated in your model.
  </li>

</ul>


#Notes
<ul>
  <li>
    When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory, and tune the learning rate accordingly.
    
    #Here we set the batch size as a function of how many GPUs we have
    BUFFER_SIZE = 10000
    BATCH_SIZE_PER_REPLICA = 64
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
  </li>

  <li>
    An OptionsDataset (and maybe other dataset types) can be shuffled and batched (and chached if memory allows) with this simply snippet:

    OptionsDataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache()

  And it becomes a BatchDataset or a cacheDataset

  </li>

  <li>
    there are different types of model callbacks

    1. TensorBoard: This callback writes a log for TensorBoard which allows you to visualize the graphs.
    2. Model Checkpoint: This callback saves the model after every epoch.
    3. Learning Rate Scheduler: Using this callback, you can schedule the learning rate to change after every epoch/batch.

  to define the directory where to store the data:

    # Define the checkpoint directory to store the checkpoints
    checkpoint_dir = './training_checkpoints'
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

  To add the callbacks:

    callbacks = [
      tf.keras.callbacks.TensorBoard(log_dir='./logs'), #logs to tensorboard
      tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,  #save the state of the model
                                        save_weights_only=True),
      tf.keras.callbacks.LearningRateScheduler(decay), #change the LR decay. 
      #decay is a function that takes epoch as argument and changes the LR depending on the epoch
      PrintLR()
    ]
  </li>

</ul>